# Riepilogo nazione

In [1]:
import altair as alt
from altair import datum
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import HTML, Markdown as md
import datetime as dt

from src.data.read_dataset import get_processed_dataset
from src.visualization import visualize as viz

df = get_processed_dataset('dpc_nazionale')

today = df.iloc[-1]
yesterday = df.iloc[-2]

In [2]:
html = '<p>Data aggiornamento: ' + str(today['data']) + '</p>'
html += viz.tabella_riassuntiva(today, yesterday)
HTML(html)

,Oggi,Variazione
Totale Casi,242363,214
Totale Positivi,13459,-136
Tamponi,5806668,52552
Casi Testati,3493126.0,29947
Deceduti,34926,12
Dimessi Guariti,193978,338
Terapia Intensiva,69,-2
Incremento,0.09%,0.01
Percentuale Positivi,6.94%,-0.05
Percentuale Nuovi Positivi,0.76%,0.09


In [3]:
resize = alt.selection_interval(bind='scales')

def rolling_avg(
    field,
    title,
    data=df,
    tooltip=["data", "nuovi_positivi", "nuovi_deceduti", 'incremento'],
):
    base = (
        alt.Chart()
        .transform_window(averaged=f"mean({field})", frame=[-4, 3])
        .encode(x="data:T", tooltip=tooltip)
        .properties(height=200, width=600,)
    )

    c1 = base.mark_line().encode(y=alt.Y("averaged:Q", title=""),)

    c2 = base.mark_bar(opacity=0.1).encode(
        y=alt.Y(f"{field}:Q", title=""),
    )

    return (
        alt.layer(c1, c2, data=data)
        .properties(title={"text": [title], "subtitle": ["Dato mediato su 7 giorni"]})
        .add_selection(
            resize
        )
    )

In [4]:
c1 = rolling_avg('nuovi_positivi', title="Nuovi positivi") 
c2 = rolling_avg('nuovi_deceduti', title="Nuovi deceduti")
c3 = rolling_avg('nuovi_tamponi', title="Nuovi tamponi")
alt.vconcat(c1, c2, c3)

alt.VConcatChart(...)

In [5]:
viz.grafici_riassuntivi(df)

alt.VConcatChart(...)